In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from scikeras.wrappers import KerasClassifier
import numpy as np

In [ ]:
df_ptbdb_normal = pd.read_csv(r"/content/drive/MyDrive/Datasets/ecg_dataset/ptbdb_normal.csv", header = None)
df_ptbdb_abnormal = pd.read_csv(r"/content/drive/MyDrive/Datasets/ecg_dataset/ptbdb_abnormal.csv", header = None)

df_ptbdb_normal.shape, df_ptbdb_abnormal.shape

((4046, 188), (10506, 188))

In [ ]:
df_ptbdb_combined = df_ptbdb_normal.append(df_ptbdb_abnormal)
df_ptbdb_combined

<ipython-input-47-b0223f8af5d4>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ptbdb_combined = df_ptbdb_normal.append(df_ptbdb_abnormal)


,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,1.000000,0.900324,0.358590,0.051459,0.046596,0.126823,0.133306,0.119125,0.110616,0.113047,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.000000,0.794681,0.375387,0.116883,0.000000,0.171923,0.283859,0.293754,0.325912,0.345083,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.909029,0.791482,0.423169,0.186712,0.000000,0.007836,0.063032,0.077002,0.074957,0.077342,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.000000,0.478893,0.056760,0.064176,0.081289,0.072732,0.055619,0.048774,0.054478,0.041643,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.000000,0.867238,0.201360,0.099349,0.141336,0.120934,0.108516,0.096393,0.093436,0.100828,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10501,0.981409,1.000000,0.559171,0.287093,0.196639,0.204862,0.215946,0.243833,0.242760,0.250268,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10502,0.906250,0.922379,0.878024,0.810484,0.712702,0.667339,0.608871,0.527218,0.480847,0.442540,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10503,1.000000,0.867971,0.674122,0.470332,0.296987,0.169307,0.077664,0.081392,0.074868,0.089779,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10504,1.000000,0.984672,0.658888,0.556394,0.446809,0.395790,0.315260,0.276367,0.261039,0.258522,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
x = df_ptbdb_combined.drop(columns=[187])
y = df_ptbdb_combined[187]

In [ ]:
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)

In [ ]:
def create_baseline_model():
    # Define the model architecture
     model = Sequential()
     model.add(Dense(187, input_shape=(187,), activation='relu'))
     model.add(Dense(1, activation='sigmoid'))

    # Compile the model
     model.compile(
        loss = tf.keras.losses.BinaryCrossentropy(),
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
        metrics = ['accuracy']
    )

     return model

estimator = KerasClassifier(model=create_baseline_model, epochs=100, batch_size=5, verbose=2)
kfold = StratifiedKFold(n_splits=10, shuffle=True)

In [ ]:
results = cross_val_score(estimator, x, encoded_Y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/100
2620/2620 - 5s - loss: 0.3919 - accuracy: 0.8157 - 5s/epoch - 2ms/step
Epoch 2/100
2620/2620 - 5s - loss: 0.3145 - accuracy: 0.8597 - 5s/epoch - 2ms/step
Epoch 3/100
2620/2620 - 6s - loss: 0.2642 - accuracy: 0.8899 - 6s/epoch - 2ms/step
Epoch 4/100
2620/2620 - 4s - loss: 0.2243 - accuracy: 0.9123 - 4s/epoch - 2ms/step
Epoch 5/100
2620/2620 - 4s - loss: 0.1966 - accuracy: 0.9253 - 4s/epoch - 2ms/step
Epoch 6/100
2620/2620 - 7s - loss: 0.1786 - accuracy: 0.9343 - 7s/epoch - 3ms/step
Epoch 7/100
2620/2620 - 4s - loss: 0.1621 - accuracy: 0.9435 - 4s/epoch - 2ms/step
Epoch 8/100
2620/2620 - 4s - loss: 0.1510 - accuracy: 0.9459 - 4s/epoch - 2ms/step
Epoch 9/100
2620/2620 - 7s - loss: 0.1387 - accuracy: 0.9525 - 7s/epoch - 3ms/step
Epoch 10/100
2620/2620 - 4s - loss: 0.1296 - accuracy: 0.9541 - 4s/epoch - 2ms/step
Epoch 11/100
2620/2620 - 4s - loss: 0.1228 - accuracy: 0.9575 - 4s/epoch - 2ms/step
Epoch 12/100
2620/2620 - 7s - loss: 0.1157 - accuracy: 0.9620 - 7s/epoch - 3ms/step
E

In [ ]:
y_pred = model.predict(X)
y_pred_classes = np.argmax(y_pred, axis= 1)

print("Classification Report:")
print(classification_report(y, y_pred_classes))

NameError: ignored